6. 신경망 학습의 개념

6.1 최적화
신경망 학습의 목적은 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것
매개변수의 최적값을 찾는 문제
신경망 최적화는 굉장히 어려운 문제
최적의 매개변수 값을 찾는 단서로 매개변수의 기울기를 이용함
매개변수의 기울기를 구해 기울어진 방향으로 매개변수 값을 갱신하는 일을 반복해서 점점
최적의 값에 다가감

6.1.2 확률적 경사 하강법
SGD는 기울어진 방향으로 일정 거리만 가겠다는 단순한 방법
update 메서드는 SGD 과정에서 반복해서 불림
params와 grads 딕셔너리 변수로 가중치 매개변수와 기울기를 저장
optimizer는 매개변수 갱신을 수행

In [ ]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

network = TwoLayerNet(...)
optimizer = SGD()

for i in range(100000):
    ..
    x_batch, t_batch = get_mini_batch(...)
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ..

SGD의 단점은 비등방성 함수에서는 탐색의 경로가 비효율적임
SGD 같이 무작정 기울어진 방향으로 진행하는 단순한 방식의 한계를 보임

6.1.4 모멘텀
v라는 변수는 물리에서 말하는 속도에 해당함
기울기 방향으로 힘을 받아 물체가 가속된다는 물리 법칙을 나타냄
모멘텀의 갱신 경로는 공이 그릇 바닥을 구르듯 움직임
SGD와 비교하면 지그재그 정도가 덜함

In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

6.1.5 AdaGrad
학습률을 정하는 효과적인 기술인 학습률 감소를 도입
학습을 진행하면서 학습률을 점차 줄여가는 방법
학습률을 서서히 낮추는 가장 간단한 방법은 매개변수 전체의 학습률 값을 일괄적으로 낮
추는 것임

In [ ]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None

    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

AdaGrad는 최솟값을 향해 효율적으로 움직임

6.1.6 Adam
모멘텀과 AdaGrad 두 방법의 이점을 조합함
Adam 갱신 과정도 모멘텀 같이 그릇 바닥을 구르듯 움직힘
모멘텀과 비슷한 패턴이지만 모멘텀 때보다 공의 좌우 흔들림이 적음

6.2 가중치의 초깃값
학습에서 특히 중요한 것이 가중치의 초깃값

6.2.1
가중치 감소 기법 - 오버피팅을 억제해 범용 성능을 높임
가중치 초깃값을 모두 0으로 설정하면? 학습이 올바로 이뤄지지 않음
그 이유는 오차역전파법에서 모든 가중치의 값이 똑같이 갱신되기 때문임
가중치가 고르게 되어버리는 상황을 막으려면 초깃값을 무작위로 설정해야 함

6.2.2 은닉층의 활성화값 분포
Xavier 초깃값은 일반적인 딥러닝 프레임워크들이 표준적으로 이용하고 있음
사용하면 앞 층에 노드가 많을수록 대상 노드의 초깃값으로 설정하는 가중치가 좁게 퍼짐
Sigmoid나 tanh에서 사용

6.2.3 ReLU를 사용할때 가중치 초깃값
ReLU에 특화된 초깃값을 He 초깃값이라고 함

6.3 배치 정규화
6.3.1 배치 정규화 알고리즘
배치 정규화의 이점
1. 학습 속도 개선
2. 초깃값에 크게 의존하지 않는다
3. 오버피팅을 억제한다 